In [ ]:
# A) EXTRACTION - etl_extract.ipynb
# 1. IMPORT LIBRARIES

import pandas as pd
import os

# Create output directory inside the ETL folder if not exists
os.makedirs("ETL_Midterm_Bricole_834/data", exist_ok=True)

import os

# 2. LOAD CSV FILES

# Load the raw data
raw_df = pd.read_csv("C:/Users/ADMIN/Desktop/ETL_Midterm_Bricole_834/raw_data.csv")

#Load the incremental data
inc_df = pd.read_csv("C:/Users/ADMIN/Desktop/ETL_Midterm_Bricole_834/incremental_data.csv")

# 3. BACKUP ORIGINALS FOR COMPARISON
raw_before = raw_df.copy()
inc_before = inc_df.copy()

# 4. PREVIEW DATA

print("RAW DATA HEAD:")
print(raw_df.head())

print("\nRAW DATA INFO:")
print(raw_df.info())

print("\nINCREMENTAL DATA HEAD:")
print(inc_df.head())

print("\nINCREMENTAL DATA INFO:")
print(inc_df.info())


# 5. OBSERVATIONS
# Missing values in raw and incremental data 
print("\nMISSING VALUES IN RAW DATA:")
print(raw_df.isnull().sum())

print("\nMISSING VALUES IN INCREMENTAL DATA:")
print(inc_df.isnull().sum())

#Duplicates in raw and incremental data
print("\nDUPLICATES IN RAW DATA:")
print(raw_df.duplicated().sum())

print("\nDUPLICATES IN INCREMENTAL DATA:")
print(inc_df.duplicated().sum())

#lOOKING FOR ANY SUSPICIOUS COLUMS AND ROWS
print("\nRAW DATA DESCRIBE:")
print(raw_df.describe(include='all'))

print("\nINCREMENTAL DATA DESCRIBE:")
print(inc_df.describe(include='all'))


# 6. SAVING RAW COPIES TO DATA DIRECTORY

raw_df.to_csv("C:/Users/ADMIN/Desktop/ETL_Midterm_Bricole_834/raw_data_copy.csv", index=False)
inc_df.to_csv("C:/Users/ADMIN/Desktop/ETL_Midterm_Bricole_834/incremental_data_copy.csv", index=False)

print("\n Raw and incremental data copied to 'ETL_Midterm_Bricole_834/data/' directory.")



In [ ]:
# B) TRANSFORM - etl_transform.ipynb


# TRANSFORMATION 1: REMOVING DUPLICATES
# To ensure clean and unique records
raw_df = raw_df.drop_duplicates()
inc_df = inc_df.drop_duplicates()

#TRANSFORMATION 2: HANDLE MISSING VALUES
# Missing values may break analysis or model training

for df in [raw_df, inc_df]:
    # Fill missing quantity with 1 (logical minimum)
    df['quantity'].fillna(1, inplace=True)
    
    # Fill missing unit_price with median value of column
    df['unit_price'].fillna(df['unit_price'].median(), inplace=True)
    
    # Fill missing customer_name and region with "Unknown"
    df['customer_name'].fillna("Unknown", inplace=True)
    df['region'].fillna("Unknown", inplace=True)

    # Drop rows with missing order_date. This is because we can't recover missing date
    df.dropna(subset=['order_date'], inplace=True)
    
#TRANSFORMATION 3: ADDING total_price
#total_price is a valuable business metric

for df in [raw_df, inc_df]:
    df['total_price'] = df['quantity'] * df['unit_price']


# 6. TRANSFORMATION 4: CONVERT order_date TO DATETIME

# Ensures dates are sortable and filterable
for df in [raw_df, inc_df]:
    df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')
    
#7. PREVIEW BEFORE & AFTER (First 10 Rows)

print("\n=== RAW DATA BEFORE TRANSFORMATION ===")
print(raw_before.head(10))

print("\n=== RAW DATA AFTER TRANSFORMATION ===")
print(raw_df.head(10))

print("\n=== INCREMENTAL DATA BEFORE TRANSFORMATION ===")
print(inc_before.head(10))

print("\n=== INCREMENTAL DATA AFTER TRANSFORMATION ===")
print(inc_df.head(10))

# 10. SAVE TRANSFORMED FILES
raw_df.to_csv("C:/Users/ADMIN/Desktop/ETL_Midterm_Bricole_834/transformed_full.csv", index=False)
inc_df.to_csv("C:/Users/ADMIN/Desktop/ETL_Midterm_Bricole_834/transformed_incremental.csv", index=False)



In [2]:
# C) LOAD - etl_load.ipynb

import pandas as pd
import os

# 1.DEFINING THE PATHS
# Location of transformed CSVs
input_path = "ETL_Midterm_Bricole_834/transformed"

# Destination for Parquet files
output_path = "ETL_Midterm_Bricole_834/parquet"
os.makedirs(output_path, exist_ok=True)

# 2.LOADING TRANSFORMED CSV FILES

# Load full transformed dataset
df_full = pd.read_csv("C:/Users/ADMIN/Desktop/ETL_Midterm_Bricole_834/transformed_full.csv")

# Load incremental transformed dataset
df_inc = pd.read_csv("C:/Users/ADMIN/Desktop/ETL_Midterm_Bricole_834/transformed_incremental.csv")

# 3. SAVE AS PARQUET

# Save to Parquet format 
df_full.to_parquet("C:/Users/ADMIN/Desktop/ETL_Midterm_Bricole_834/full_data.parquet", index=False)
df_inc.to_parquet("C:/Users/ADMIN/Desktop/ETL_Midterm_Bricole_834/incremental_data.parquet", index=False)


# 4. CONFIRMATION

print("✅ Parquet files saved to:", output_path)

✅ Parquet files saved to: ETL_Midterm_Bricole_834/parquet
